In [ ]:
# dynetx сан суулгах
!pip install dynetx

In [ ]:
# Шаардлагатай сангуудыг импортлох
import dynetx as dn  # Динамик сүлжээний шинжилгээний сан
import networkx as nx  # Сүлжээний шинжилгээний үндсэн сан
import random  # Санамсаргүй тоо үүсгэх сан

# CSV файлаас сүлжээг уншиж авах функц
def read_net(filename):
    g = nx.Graph()  # Шинэ граф үүсгэх
    with open(filename) as f:
        f.readline()  # Эхний мөрийг алгасах (column headers)
        for l in f:
            l = l.split(",")  # Мөрийг таслалаар салгах
            g.add_edge(l[0], l[1])  # Ирмэг нэмэх
    return g

# Динамик граф үүсгэх
g = dn.DynGraph()

In [ ]:
# Game of Thrones цувралын 8 улирлын өгөгдлийг ачаалах
for t in range(1, 9):
    # t улирал бүрийн холбоосын файлыг уншиж авах
    er = read_net(f'/content/sample_data/got-s{t}-edges.csv')
    # Динамик графт холбоосуудыг цаг хугацааны мэдээлэлтэй нэмэх
    g.add_interactions_from(er.edges, t=t)

In [ ]:
# Бүх цаг хугацааны үеийн дугааруудыг харуулах
g.temporal_snapshots_ids()

[1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
# Зөвхөн 1-р үеийн графыг авах
g1 = g.time_slice(1)

In [ ]:
# g1 графын төрөл, оройн тоо, ирмэгийн тоог харуулах
type(g1), g1.number_of_nodes(), g1.number_of_edges()

(dynetx.classes.dyngraph.DynGraph, 126, 549)

In [ ]:
# 0-3 үеийн графыг авах
g0_3 = g.time_slice(0, 3)

In [ ]:
# g0_3 графын төрөл, оройн тоо, ирмэгийн тоог харуулах
type(g0_3), g0_3.number_of_nodes(), g0_3.number_of_edges()

(dynetx.classes.dyngraph.DynGraph, 237, 1182)

In [ ]:
# g1 графыг хавтгай граф болгон хувиргах
g1_flat = nx.Graph(g1.edges())

In [ ]:
# g1_flat графын төрөл, оройн тоо, ирмэгийн тоог харуулах
type(g1_flat), g1_flat.number_of_nodes(), g1_flat.number_of_edges()

(networkx.classes.graph.Graph, 126, 549)

In [ ]:
# Бүх холбоосуудын хоорондын цаг хугацааны зайг тооцоолох
r = g.inter_event_time_distribution()
print(f"Number interactions: temporal distance\t{r}")

Number interactions: temporal distance	{0: 3307, 1: 8}


In [ ]:
# ARYA зангилааны холбоосуудын хоорондын цаг хугацааны зайг тооцоолох
r = g.inter_event_time_distribution("ARYA")
print(f"Number interactions: temporal distance\t{r}")

Number interactions: temporal distance	{0: 137, 1: 8}


In [ ]:
# JON болон ARYA зангилаануудыг тодорхойлох
u = 'JON'
v = 'ARYA'

In [ ]:
# JON болон ARYA зангилаануудын хоорондын цаг хугацааны зайг тооцоолох
r = g.inter_event_time_distribution(u, v)
print(f"Number interactions: temporal distance\t{r}")

Number interactions: temporal distance	{0: 2, 6: 1, 1: 1}


In [ ]:
# ARYA зангилааны 2-р үеийн зэрэг
g.degree(t=2)['ARYA']

27

In [ ]:
# Графын хамрах хүрээ
g.coverage()

0.2977216748768473

In [ ]:
# BERIC зангилааны хувь нэмэр
g.node_contribution("BERIC")

0.625

In [ ]:
# JON болон ARYA зангилаануудын ирмэгийн хувь нэмэр
g.edge_contribution(u, v)

0.625

In [ ]:
# JON болон ARYA зангилаануудын хосын жигд байдал
g.node_pair_uniformity(u, v)

1.0

In [ ]:
# Графын нягтшил
g.density()

0.06686633244351846

In [ ]:
# JON зангилааны нягтшил
g.node_density(u)

0.2295760082730093

In [ ]:
# JON болон ARYA зангилаануудын хосын нягтшил
g.pair_density(u, v)

### Графын нягтшил (Snapshot Density)

In [ ]:
# Цаг хугацаа бүрийн графын нягтшилыг тооцоолох
for t in g.temporal_snapshots_ids():
    print(f"{t}\t{g.snapshot_density(t)}")

1	0.06971428571428571
2	0.05886627906976744
3	0.06608969315499606
4	0.04535563715490276
5	0.05640222190571144
6	0.05404055538907202
7	0.1271604938271605
8	0.20473898556090336


### Зам шинжилгээ (Path Analysis)

In [ ]:
# Замын алгоритмуудыг импортлох
import dynetx.algorithms as al
# GENDRY-ээс GREY_WORM хүртэлх цаг хугацаанд хамааралтай замуудыг олох (1-5 үе хооронд)
paths = al.time_respecting_paths(g, "GENDRY", "GREY_WORM", start=1, end=5)

In [ ]:
# Олдсон замын жишээ. Зам бүр нь (from_node, to_node, time) хэлбэртэй tuple байна
p = paths[0]
p

[]

### Daalgavar

In [ ]:
# Олдсон замуудыг тэмдэглэгээтэй болгох (хамгийн богино, хамгийн хурдан гэх мэт)
annotated = al.annotate_paths(paths[("GENDRY", "GREY_WORM")])

In [ ]:
# Хамгийн богино зам харуулах
annotated['shortest']

In [ ]:
# Хамгийн хурдан зам харуулах
annotated['fastest']

In [ ]:
# Богино бөгөөд хурдан зам
annotated['shortest_fastest']

In [ ]:
# Хурдан бөгөөд богино зам
annotated['fastest_shortest']

In [ ]:
# Хамгийн эрт зам
annotated['foremost']